# **Imports**

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
#import urllib3.request,sys,time
from urllib.error import HTTPError
from urllib.request import urlopen
import pandas as pd
import time
import warnings
import json
#from google.colab import files
warnings.filterwarnings('ignore')

# Treatment functions

In [2]:
def getNews(article_url):
    try:
        html = urlopen(article_url)
    except HTTPError as e:
        print(article_url)
        return None
    try :
        bsObj = BeautifulSoup(html.read())
        article=dict()
    except :
        return None
    try:
        article["category"]=bsObj.find("aside",{'data-gu-name':{'title'}}).text
    except AttributeError as e:
        article["category"]=''
    try:
        article["keywords"]=bsObj.find("div",{'class':{'dcr-lwa3gj'}}).text
    except AttributeError as e:
        article["keywords"]=''
    try:
        article["author"]=bsObj.find("a",{'rel':{'author'}}).text
    except AttributeError as e:
        article["author"]=''
    try:
        article["headline"]=bsObj.find("h1",{'data-gu-name':{'headline'}}).text
    except AttributeError as e:
        try:
            article["headline"]=bsObj.find("h1",{'itemprop':{'headline'}}).text
        except AttributeError as e:
            article["headline"]=''
    try:
        article["summary"]=bsObj.find("div",{'class':{'content__standfirst'}}).text
    except AttributeError as e:
        try:
            article["summary"]=bsObj.find("div",{'data-gu-name':{'standfirst'}}).text
        except AttributeError as e:
            article["summary"]=''
    try:
        article["fulltext"]=bsObj.find("div",{'id':{'maincontent'}}).text
    except AttributeError as e:
        try:
            article["fulltext"]=bsObj.find("div",{'class':{'from-content-api podcast__body'}}).text
        except AttributeError as e:
            try:
                article["fulltext"]=bsObj.find("div",{'itemprop':{'articleBody'}}).text
            except AttributeError as e:
                article["fulltext"]= ''
    article["url"]= article_url
    time.sleep(1)
    return article

# API Test with Facebook articles

In [45]:
url='https://content.guardianapis.com/search?page=1&q=facebook&from-date=2010-01-01&api-key=38f1325b-db6b-4271-8110-0a8f2fd33507'
response=requests.get(url)

In [46]:
response

<Response [200]>

In [47]:
response.json()['response']['status'] != 'error' #['results'][0]['webUrl']

True

In [48]:
response.json()['response']['results'][9]['webUrl']

'https://www.theguardian.com/technology/2023/may/08/facebook-founder-mark-zuckerberg-jiu-jiutsu'

In [ ]:
for page_num in range(1,10): #doit aller jusqu'à 4000 pour Facebook
    url = 'https://content.guardianapis.com/search?page='+str(page_num)+'&q=Facebook&from-date=2000-01-01&api-key=3a213067-3316-4156-ae7a-1bf5c7b3e834'
    response=requests.get(url)
    if response.json()['response']['status'] != 'error' :
        for i in range(0,10) :
            url_list.append(response.json()['response']['results'][i]['webUrl'])

In [ ]:
df_links = pd.DataFrame(url_list, columns=['url'])

In [ ]:
df_links

,url
0,https://www.theguardian.com/business/2023/may/...
1,https://www.theguardian.com/technology/2023/ma...
2,https://www.theguardian.com/technology/2023/ma...
3,https://www.theguardian.com/technology/2023/ap...
4,https://www.theguardian.com/technology/2023/ju...
...,...
235,https://www.theguardian.com/commentisfree/2023...
236,https://www.theguardian.com/technology/2022/no...
237,https://www.theguardian.com/world/2023/may/10/...
238,https://www.theguardian.com/world/2022/apr/18/...


# Treatment functions 2

On crée nos deux fonctions systématiques de traitement pour The Guardian, avec en paramètres le thème que l'on veut considérer, et le nombre de pages que l'on va prendre (la page max dépend du thème, c'est environ 4000 pour Facebook).

In [3]:
articles_list = []

In [4]:
def articles_GUA_from_links(df_links):
    articles = []
    for ind in df_links.index:
        if getNews(df_links.iloc[ind,0]) is not None :
          try:
            news = getNews(df_links.iloc[ind,0])
            articles.append(news)
            articles_list.append(news)
          except :
            articles = articles
    #print(articles, 'articles')
    return articles

In [12]:
def articles_from_base_url_GUA(theme, pagemax) :
    url_list = []
    for page_num in range(918,pagemax): #doit aller jusqu'à 3103 pour Facebook, jusqu'à 2010. Commencer à 1 !!!
      url = 'https://content.guardianapis.com/search?page='+str(page_num)+'&q='+str(theme)+'&from-date=2010-01-01&api-key=38f1325b-db6b-4271-8110-0a8f2fd33507'
      response=requests.get(url)
      try :
        if response.json()['response']['status'] == 'ok' :
          for i in range(0,10) :
            url_list.append(response.json()['response']['results'][i]['webUrl'])
      except :
        url_list = url_list
      #time.sleep(1)
      if url_list == [] :
        return (page_num, 'page problèmatique')
      if page_num%100 == 0 :
        print(page_num)
      #print(url_list)
    df_links = pd.DataFrame(url_list, columns=['url'])
    #print(df_links)
    news  = articles_GUA_from_links(df_links)
    #print(news, 'news')
    df_news = pd.DataFrame(news)
    #print(df_news)
    return df_news

# Dataframe Facebook

rangemax = 3103

In [113]:
df_news_fb = articles_from_base_url_GUA('Facebook', 3103) # il faut aller à 3103

In [31]:
len(articles_list)

6770

# Travail sur articles_list

In [32]:
with open("liste_temporaire_3_DE_GUA.json", "w") as fp:
  json.dump(articles_list, fp)

#files.download("liste_temporaire_2_DE_GUA")

**On récupère la liste temporaire, et on divise par 10 pour savoir où reprendre le scraping**

In [25]:
with open('liste_temporaire_1_fb_GUA.json') as f:
    liste_temporaire_1 = json.load(f)

In [26]:
with open('liste_temporaire_2_fb_GUA.json') as f:
    liste_temporaire_2 = json.load(f)

In [27]:
with open('liste_temporaire_3_fb_GUA.json') as f:
    liste_temporaire_3 = json.load(f)

In [28]:
len(liste_temporaire_1) + len(liste_temporaire_2) + len(liste_temporaire_3)

26622

On reconstitue le DataFrame global avec tous les articles :

In [29]:
liste_articles = liste_temporaire_1 + liste_temporaire_2 + liste_temporaire_3

In [30]:
len(liste_articles)

26622

On regarde les articles problématiques pour ranger cette liste sous forme de dataframe

In [31]:
for t in range(5):
    for i in liste_articles:
        try:
            i.keys()
        except:
            liste_articles.remove(i)

In [98]:
len(liste_articles)

26578

In [32]:
df_fb_GUA = pd.DataFrame(liste_articles)

In [33]:
df_fb_GUA = df_fb_GUA[df_fb_GUA['url'].str.contains('meta|facebook|messenger|instagram|oculus|whatsApp|zuckerberg|olivan|clegg|social-media|gafa|gafam|big-tech|consumer-privacy|tech')]

In [101]:
len(df_fb_GUA)

5429

On perd beaucoup quand même ! 

In [34]:
#from google.colab import files
df_fb_GUA.to_csv('df_fb_GUA.csv')
#files.download('df_fb_GUA.csv')

# Dataframe Amazon

rangemax = 2082

In [13]:
articles_list = []

In [14]:
df_news_am = articles_from_base_url_GUA('Amazon', 2082)

1900
2000
https://www.theguardian.com/film/2010/apr/09/inglourious-basterds-inspiration-dvd-hit
https://www.theguardian.com/business/2010/aug/21/ebay-businesses-turnover-rises
https://www.theguardian.com//Smarthealthcare/patient-smart-healthcare-live-telecare-email-07jul10


# Travail avec articles_list

In [8]:
len(articles_list)

5264

In [20]:
with open("liste_temporaire_4_am_GUA.json", "w") as fp:
  json.dump(articles_list, fp)

In [21]:
with open('liste_temporaire_1_am_GUA.json') as f:
    liste_temporaire_1 = json.load(f)

with open('liste_temporaire_2_am_GUA.json') as f:
    liste_temporaire_2 = json.load(f)

with open('liste_temporaire_3_am_GUA.json') as f:
    liste_temporaire_3 = json.load(f)

In [22]:
len(liste_temporaire_1) + len(liste_temporaire_2) + len(liste_temporaire_3)

18107

# Synthèse

In [23]:
liste_articles = liste_temporaire_1 + liste_temporaire_2 + liste_temporaire_3

In [25]:
len(liste_articles)

18107

In [26]:
for t in range(5):
    for i in liste_articles:
        try:
            i.keys()
        except:
            liste_articles.remove(i)

In [41]:
df_am_GUA = pd.DataFrame(liste_articles)

In [42]:
df_am_GUA = df_am_GUA[df_am_GUA['url'].str.contains('amazon|prime|whole-foods|zappos|pillpack|twitch|audible|goodreads|imdb|bezos|jassy|olsavsky|alexander|gafa|gafam|big-tech|consumer-privacy|tech')]

In [43]:
yy=df_am_GUA[df_am_GUA['fulltext'].str.contains('forest|tribe|jungle|swallows-and-amazon|amazons')]
df_am_GUA.drop(yy.index,inplace=True)

In [ ]:
df_am_GUA

In [45]:
#from google.colab import files
df_am_GUA.to_csv('df_am_GUA.csv')
#files.download('df_am_GUA.csv')

# Dataframe Apple

rangemax = 2732

In [6]:
articles_list = []

In [7]:
df_news_ap = articles_from_base_url_GUA('Apple', 2732)

1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
https://www.theguardian.com/film/2015/nov/02/new-movies-from-sandra-bullock-and-bradley-cooper-bomb-at-us-box-office
https://www.theguardian.com/travel/2016/feb/27/five-alls-the-plough-cotswolds-hotel-pub-review


In [13]:
len(articles_list)

598

# Travail avec articles_list

In [15]:
len(articles_list)

598

In [17]:
with open("liste_temporaire_3_ap_GUA.json", "w") as fp:
  json.dump(articles_list, fp)

In [19]:
with open('liste_temporaire_1_ap_GUA.json') as f:
    liste_temporaire_1 = json.load(f)

with open('liste_temporaire_2_ap_GUA.json') as f:
    liste_temporaire_2 = json.load(f)

with open('liste_temporaire_3_ap_GUA.json') as f:
    liste_temporaire_3 = json.load(f)


len(liste_temporaire_1) + len(liste_temporaire_2) + len(liste_temporaire_3) 

14268

# Synthèse

In [20]:
liste_articles = liste_temporaire_1 + liste_temporaire_2 + liste_temporaire_3

In [21]:
for t in range(5):
    for i in liste_articles:
        try:
            i.keys()
        except:
            liste_articles.remove(i)

In [22]:
df_ap_GUA = pd.DataFrame(liste_articles)

In [23]:
df_ap_GUA = df_ap_GUA[df_ap_GUA['url'].str.contains('apple|iphone|iPad|mac|watch|macbook|ios|airpods|app-store|itunes|icloud|beats|siri|shazam|cook|jobs|gafa|gafam|big-tech|consumer-privacy|tech')]

In [24]:
yy=df_ap_GUA[df_ap_GUA['fulltext'].str.contains('apples|pie|pies|recipe|tart|crumble|bake|vinegar')]
df_ap_GUA.drop(yy.index,inplace=True)

In [26]:
df_ap_GUA.to_csv('df_ap_GUA.csv')

# Dataframe Google

rangemax = 3103

In [21]:
articles_list = []

In [22]:
df_news_go = articles_from_base_url_GUA('Google', 3103)

2400
2500
2600
2700
2800
2900
3000
3100
https://www.theguardian.com/higher-education-network/blog/2012/mar/15/ebooks-academic-future-universities-steven-schwartz
https://www.theguardian.com//Public/public-sector-websites-usability
https://www.theguardian.com//Smarthealthcare/microsoft-healthvault-uk-open-wellness-cameron-22jun10
https://www.theguardian.com//Public/government-data-activate-summit
https://www.theguardian.com//Smarthealthcare/white-paper-unravel-summary-care-records-19jul10
https://www.theguardian.com//Public/cyber-attack-public-sector-comment
https://www.theguardian.com//Smarthealthcare/cameron-clegg-conservatives-libdem-nhs-npfit-12may10
https://www.theguardian.com//Smarthealthcare/information-security-ico-david-smith-nhs-05may10
https://www.theguardian.com//Smarthealthcare/patient-13jan10
https://www.theguardian.com//Smarthealthcare/free-data-gov-uk-open-access-03mar10


# Travail avec articles_list

In [23]:
with open("liste_temporaire_3_go_GUA.json", "w") as fp:
  json.dump(articles_list, fp)

In [16]:
len(articles_list)

8465

In [25]:
with open('liste_temporaire_1_go_GUA.json') as f:
    liste_temporaire_1 = json.load(f)
with open('liste_temporaire_2_go_GUA.json') as f:
    liste_temporaire_2 = json.load(f)
with open('liste_temporaire_3_go_GUA.json') as f:
    liste_temporaire_3 = json.load(f)

# Synthèse

In [30]:
liste_articles = liste_temporaire_1 + liste_temporaire_2 + liste_temporaire_3

In [31]:
for t in range(5):
    for i in liste_articles:
        try:
            i.keys()
        except:
            liste_articles.remove(i)

In [32]:
df_ap_GUA = pd.DataFrame(liste_articles)

In [34]:
df_go_GUA = df_ap_GUA [df_ap_GUA ['url'].str.contains('google|android|chrome|gmail|maps|playstore|pixel|waze|youTube|alphabet|mandiant|fitbit|looker|nest|doubleclick|page|brin|pichai|kurian|gafa|gafam|big-tech|consumer-privacy|tech')]

In [37]:
df_go_GUA.to_csv('df_go_GUA.csv')

# Dataframe Microsoft

rangemax = 932

In [13]:
articles_list = []

In [14]:
df_news_mi = articles_from_base_url_GUA('Microsoft', 932)

https://www.theguardian.com//Smarthealthcare/cameron-clegg-conservatives-libdem-nhs-npfit-12may10
https://www.theguardian.com//Smarthealthcare/zafar-chaudry-liverpool-womens-alder-hey-31mar10
https://www.theguardian.com//Smarthealthcare/patient-13jan10
https://www.theguardian.com//Smarthealthcare/clinical-dashboards-bolton-homerton-salisbury-christie-14apr10


# Travail avec articles_list

In [15]:
len(articles_list)

136

In [16]:
with open("liste_temporaire_4_mi_GUA.json", "w") as fp:
  json.dump(articles_list, fp)

In [17]:
with open('liste_temporaire_1_mi_GUA.json') as f:
    liste_temporaire_1 = json.load(f)
with open('liste_temporaire_2_mi_GUA.json') as f:
    liste_temporaire_2 = json.load(f)
with open('liste_temporaire_3_mi_GUA.json') as f:
    liste_temporaire_3 = json.load(f)
with open('liste_temporaire_4_mi_GUA.json') as f:
    liste_temporaire_4 = json.load(f)

In [18]:
len(liste_temporaire_1)+len(liste_temporaire_2)+len(liste_temporaire_3)+len(liste_temporaire_4)

9316

# Synthèse

In [19]:
liste_articles = liste_temporaire_1 + liste_temporaire_2 + liste_temporaire_3 + liste_temporaire_4

In [20]:
for t in range(5):
    for i in liste_articles:
        try:
            i.keys()
        except:
            liste_articles.remove(i)

In [21]:
df_mi_GUA = pd.DataFrame(liste_articles)

In [22]:
df_mi_GUA = df_mi_GUA[df_mi_GUA['url'].str.contains('microsoft|windows|cortana|excel|explorer|office|edge|teams|outlook|powerpoint|skype|surface|word|xbox|linkedIn|github|mojang|gates|nadella|gafa|gafam|big-tech|consumer-privacy|tech')]

In [23]:
df_mi_GUA.to_csv('df_mi_GUA.csv')

# Dataframe final

In [47]:
df_fb_GUA = pd.read_csv('df_fb_GUA.csv', index_col=0)
df_am_GUA = pd.read_csv('df_am_GUA.csv', index_col=0)
df_ap_GUA = pd.read_csv('df_ap_GUA.csv', index_col=0)
df_go_GUA = pd.read_csv('df_go_GUA.csv', index_col=0)
df_mi_GUA = pd.read_csv('df_mi_GUA.csv', index_col=0)

Il faut supprimer les lignes en double !!!

In [50]:
df_final = pd.concat([df_fb_GUA, df_am_GUA, df_ap_GUA, df_go_GUA, df_mi_GUA]).reset_index()
df_final.drop_duplicates(subset ="url", keep = 'first', inplace=True)
df_final.to_csv('articlesGUA.csv', index=False)
#files.download('articlesGUA.csv')

In [ ]:
files.download('articlesGUA.csv')

In [ ]:
while True:pass